# Intro

This notebook creates the view for sa_usq which Census will use for the sync


## Change History

<style>
  table {margin-left: 0 !important;}
</style>

| Date    | Author | Description |
| :-------- | :------- | :------- | 
|2025-03-14 | Mclain R |  Created Date|
|2025-04-17 | Mclain R |  Change to dynamically get variables to format names|
|2025-04-22 | Mclain R |  Change to dynamically get round # instead of dates|

code for single select import sa rae fields

adding a trim to the email in cte1 and other ctes using cte1.user_email

In [ ]:
CREATE OR ALTER PROCEDURE sp_create_view_v_sausq
AS
BEGIN
    DECLARE @year NVARCHAR(4) = FORMAT(GETDATE(), 'yyyy');
    DECLARE @month_code NVARCHAR(2) = FORMAT(GETDATE(), 'MM');
    -- DECLARE @date_code NVARCHAR(8) = FORMAT(GETDATE(), 'yyyyMMdd');
    -- DECLARE @date_code NVARCHAR(8) = FORMAT(DATEADD(DAY, -1, GETDATE()), 'yyyyMMdd'); -- specific date
    DECLARE @cohort_prefix NVARCHAR(50) = CONCAT(@year, ' ', @month_code, ' SA Qualifier');
    DECLARE @revision NVARCHAR(10);
    SELECT TOP 1 @revision = revision
    FROM bronze_lakehouse.dbo.sausq_revision_tracker
    ORDER BY created_on DESC;

    DECLARE @sql NVARCHAR(MAX) = N'';

    SET @sql += N'CREATE OR ALTER VIEW v_sausq AS ';
    SET @sql += N'
    WITH cte1 AS (
        SELECT *, CONCAT(current_volume, trim(lower(user_email))) AS combined
        FROM [silver_lakehouse].[dbo].[nucleus__sa_usq]
        WHERE trim(lower(user_email)) IS NOT NULL AND current_volume IN (0,1)
    ),';

    SET @sql += N'
    cte2 AS (
        SELECT DISTINCT combined FROM cte1
    ),';

    SET @sql += N'
    cte3 AS (
        SELECT combined, RIGHT(combined, LEN(combined) - 1) AS [unique]
        FROM cte2
    ),';

    SET @sql += N'
    cte4 AS (
        SELECT 
            cte1.combined,
            STRING_AGG(
                ''<li style="color:#650360; mso-line-height-rule:exactly; font-size:14px; font-weight:400; font-family:Helvetica,Arial,sans-serif;">''
                + CAST(cte1.dealername AS NVARCHAR(MAX)) +
                ''</li>'',
                ''''
            ) AS dealer
        FROM cte1
        GROUP BY cte1.combined
    ),';

    SET @sql += N'
    cte5 AS (
        SELECT 
            cte3.*, 
            COALESCE(cte4.dealer, '''') AS dealer,
            LEFT(cte3.combined, 1) AS vol
        FROM cte3
        LEFT JOIN cte4 ON cte3.combined = cte4.combined
    ),';

    SET @sql += N'
    cte6 AS (
        SELECT *,
            CONCAT(
                ''<ul style="list-style:disc; color:#650360; mso-line-height-rule:exactly; font-size:14px; font-weight:400; font-family:Helvetica,Arial,sans-serif;">'',
                dealer,
                ''</ul>''
            ) AS sa_usq_list,
            CASE
                WHEN vol = 0 THEN ''' + @cohort_prefix + ': '  + @revision + ' - 00''
                WHEN vol = 1 THEN ''' + @cohort_prefix + ': '  + @revision + ' - 01''
                ELSE NULL
            END AS cohort,
            CASE
                WHEN vol = 0 THEN ''Vol_00_' + @revision + '''
                WHEN vol = 1 THEN ''Vol_01_' + @revision + '''
                ELSE NULL
            END AS vol2
        FROM cte5
    ),';

    SET @sql += N'
    cte7 AS (
        SELECT *,
            CASE WHEN vol = 0 THEN sa_usq_list ELSE NULL END AS sa_usq_list_00,
            CASE WHEN vol = 1 THEN sa_usq_list ELSE NULL END AS sa_usq_list_01
        FROM cte6
    ),';

    SET @sql += N'
    cte8 AS (
        SELECT DISTINCT
            cte7.[unique] AS email,
            JSON_ARRAYAGG(cte7.cohort) AS email_campaign_cohort,
            JSON_ARRAYAGG(cte7.vol2) AS sa_usq_email_version,
            STRING_AGG(CAST(cte7.sa_usq_list_00 AS NVARCHAR(MAX)), '';'' ) AS sa_usq_list_00,
            STRING_AGG(CAST(cte7.sa_usq_list_01 AS NVARCHAR(MAX)), '';'' ) AS sa_usq_list_01
        FROM cte7
        GROUP BY cte7.[unique]
    ),';

    SET @sql += N'
    cte9 AS (
        SELECT
            cte8.email,
            COALESCE(cte8.email_campaign_cohort, ''JSON_ARRAY()'') AS email_campaign_cohort,
            COALESCE(cte8.sa_usq_email_version, ''JSON_ARRAY()'') AS sa_usq_email_version,
            JSON_ARRAY(''' + @cohort_prefix + ''') AS email_campaign_inclusion,
            JSON_ARRAY(''Yes'') AS sa_usq_new_upload,
            COALESCE(cte8.sa_usq_list_00, '''') AS sa_usq_list_00,
            COALESCE(cte8.sa_usq_list_01, '''') AS sa_usq_list_01
        FROM cte8
    ),';

    SET @sql += N'
    cte10 AS (
        SELECT 
            trim(lower(cte1.user_email)) AS email,
            cte1.name_for_mythic AS import_sa_rae_full_name,
            cte1.rae_num AS import_sa_rae_phone_number,
            cte1.rae_email AS import_sa_rae_ally_email,
            REPLACE(cte1.rae_email, ''@ally.com'', ''@dealerbeyond.ally.com'') AS import_sa_rae_hubspot_email,
            cte1.rae_title AS import_sa_rae_title,
            ROW_NUMBER() OVER (PARTITION BY trim(lower(cte1.user_email)) ORDER BY cte1.rae_email DESC) AS row_num
        FROM cte1
    ),';

    SET @sql += N'
    cte_final AS (
        SELECT * FROM cte10 WHERE row_num = 1
    ),';

    SET @sql += N'
    cte_final2 AS (
        SELECT 
            cte9.*, 
            COALESCE(cte_final.import_sa_rae_full_name, '''') AS import_sa_rae_full_name,
            COALESCE(cte_final.import_sa_rae_phone_number, '''') AS import_sa_rae_phone_number,
            COALESCE(cte_final.import_sa_rae_ally_email, '''') AS import_sa_rae_ally_email,
            COALESCE(cte_final.import_sa_rae_hubspot_email, '''') AS import_sa_rae_hubspot_email,
            COALESCE(cte_final.import_sa_rae_title, '''') AS import_sa_rae_title
        FROM cte9
        LEFT JOIN cte_final ON cte9.email = cte_final.email
    )

    SELECT
        email,
        email_campaign_cohort,
        sa_usq_email_version,
        email_campaign_inclusion,
        sa_usq_new_upload,
        sa_usq_list_00,
        sa_usq_list_01,
        CASE
            WHEN import_sa_rae_full_name LIKE ''%CC'' THEN ''Izaiah Cruz''
            WHEN import_sa_rae_full_name LIKE ''Open%'' THEN ''''
            ELSE import_sa_rae_full_name
        END AS import_sa_rae_full_name,
        import_sa_rae_phone_number,
        import_sa_rae_ally_email,
        import_sa_rae_hubspot_email,
        import_sa_rae_title
    FROM cte_final2;
    ';

    EXEC sp_executesql @sql;
END;


In [ ]:
EXEC sp_create_view_v_sausq;


testing

In [ ]:
SELECT TOP 1 revision
    FROM bronze_lakehouse.dbo.sausq_revision_tracker

In [ ]:
SELECT * FROM v_sausq
where LEN(sa_usq_list_00) > 330
order by LEN(sa_usq_list_00) desc

In [ ]:
SELECT * from v_sausq
where email like '%globalresponse.com%' or email like '%doohen@ally%'

In [ ]:
-- OLD PROCESS BEFORE MAKING DYNAMIC

--CREATE OR ALTER VIEW v_sausq AS

-- create new field combined
WITH cte1 AS (
    SELECT
        *,
        concat(current_volume,user_email) AS combined
    FROM [silver_lakehouse].[dbo].[nucleus__sa_usq]
    WHERE user_email IS NOT NULL
        AND current_volume IN (0,1)
),

-- distinct combined email
cte2 AS (
    SELECT DISTINCT
        combined
    FROM cte1
),

-- extract email
cte3 AS (
    SELECT
        combined,
        RIGHT(combined, LEN(combined) - 1) AS [unique]
    FROM cte2
),

-- add html code and create dealer field
cte4 AS (
    SELECT 
        cte1.combined,
        STRING_AGG(CAST(cte1.dealername AS NVARCHAR(MAX)), '</li><li style="color:#650360; mso-line-height-rule:exactly; font-size:14px; font-weight:400; font-family:Helvetica,Arial,sans-serif;">') AS dealer
    FROM cte1
    GROUP BY cte1.combined
),

-- create vol field
cte5 AS (
    SELECT 
        cte3.*, 
        COALESCE(cte4.dealer, '') AS dealer,
        LEFT(cte3.combined, 1) AS vol
    FROM cte3
    LEFT JOIN cte4
        ON cte3.combined = cte4.combined
),

-- create cohort and vol2 fields
cte6 AS (
    SELECT
        *,
        CONCAT('<ul style="list-style:disc ;color:#650360; mso-line-height-rule:exactly; font-size:14px; font-weight:400; font-family:Helvetica,Arial,sans-serif;"><li style="color:#650360; mso-line-height-rule:exactly; font-size:14px; font-weight:400; font-family:Helvetica,Arial,sans-serif;">',dealer,'</li></ul></span></p>') AS sa_usq_list,
        CASE
            WHEN vol = 0 THEN '2025 04 SA Qualifier: 00'
            WHEN vol = 1 THEN '2025 04 SA Qualifier: 01'
            ELSE NULL
        END AS cohort,
        CASE
            WHEN vol = 0 THEN 'VOL_00'
            WHEN vol = 1 THEN 'VOL_01'
            ELSE NULL
        END AS vol2
    FROM cte5
),

-- create sa_usq list fields
cte7 AS (
    SELECT
        *,
        CASE
            WHEN vol = 0 THEN sa_usq_list
            ELSE NULL
        END AS sa_usq_list_00,
        CASE
            WHEN vol = 1 THEN sa_usq_list
            ELSE NULL
        END AS sa_usq_list_01
    FROM cte6
),

-- make fields multi-select
cte8 AS (
    SELECT DISTINCT
        cte7.[unique] AS email,
        JSON_ARRAYAGG(cte7.cohort) AS email_campaign_cohort,
        JSON_ARRAYAGG(cte7.vol2) AS sa_usq_email_version,
        STRING_AGG(CAST(cte7.sa_usq_list_00 AS NVARCHAR(MAX)), ';') AS sa_usq_list_00,
        STRING_AGG(CAST(cte7.sa_usq_list_01 AS NVARCHAR(MAX)), ';') AS sa_usq_list_01
    FROM cte7
    GROUP BY cte7.[unique]
),

-- create new fields
cte9 AS (
    SELECT
        cte8.email,
        COALESCE(cte8.email_campaign_cohort, 'JSON_ARRAY()') AS email_campaign_cohort,
        COALESCE(cte8.sa_usq_email_version, 'JSON_ARRAY()') AS sa_usq_email_version,
        JSON_ARRAY('2025 04 SA Qualifier') AS email_campaign_inclusion,
        JSON_ARRAY('Yes') AS sa_usq_new_upload,
        COALESCE(cte8.sa_usq_list_00,'') AS sa_usq_list_00,
        COALESCE(cte8.sa_usq_list_01,'') AS sa_usq_list_01
    FROM cte8
),

-- grab only 1 rae for each field
cte10 AS (
    SELECT 
        cte1.user_email AS email,
        cte1.name_for_mythic AS import_sa_rae_full_name,
        cte1.rae_num AS import_sa_rae_phone_number,
        cte1.rae_email AS import_sa_rae_ally_email,
        REPLACE(cte1.rae_email,'@ally.com','@dealerbeyond.ally.com') AS import_sa_rae_hubspot_email,
        cte1.rae_title AS import_sa_rae_title,
        ROW_NUMBER() OVER (PARTITION BY cte1.user_email ORDER BY cte1.rae_email DESC) AS row_num
    FROM cte1
),

cte_final AS (
    SELECT *
    FROM cte10
    WHERE row_num = 1
),

--final select
cte_final2 AS (
SELECT 
    cte9.*, 
    COALESCE(cte_final.import_sa_rae_full_name, '') AS import_sa_rae_full_name,
    COALESCE(cte_final.import_sa_rae_phone_number, '') AS import_sa_rae_phone_number,
    COALESCE(cte_final.import_sa_rae_ally_email, '') AS import_sa_rae_ally_email,
    COALESCE(cte_final.import_sa_rae_hubspot_email, '') AS import_sa_rae_hubspot_email,
    COALESCE(cte_final.import_sa_rae_title, '') AS import_sa_rae_title
FROM cte9
LEFT JOIN cte_final ON cte9.email = cte_final.email
)

SELECT
    email,
    email_campaign_cohort,
    sa_usq_email_version,
    email_campaign_inclusion,
    sa_usq_new_upload,
    sa_usq_list_00,
    sa_usq_list_01,
    CASE
        WHEN import_sa_rae_full_name like '%CC' THEN 'Izaiah Cruz'
        WHEN import_sa_rae_full_name like 'Open%' THEN ''
        ELSE import_sa_rae_full_name
    END AS import_sa_rae_full_name,
    import_sa_rae_phone_number,
    import_sa_rae_ally_email,
    import_sa_rae_hubspot_email,
    import_sa_rae_title
FROM cte_final2

In [ ]:
SELECT distinct import_sa_rae_full_name FROM v_sausq
where import_sa_rae_full_name like '%CC%'

In [ ]:
SELECT distinct import_sa_rae_full_name FROM v_sausq
where import_sa_rae_full_name like '%Open%'